In [4]:
import json, codecs
fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json'
db_schema={}
with codecs.open(fpath, 'r', 'utf-8-sig') as f:
    data = json.load(f)
    for db, db_info in data.items():
        tables=db_info['tag_names_original']
        cols = db_info['property_names']
        if db not in db_schema:
            db_schema[db]=[(db, 'has', '*')]
        for table in tables:
            db_schema[db].append((db, 'has', '{}.{}'.format(db,table )))
        
        for col in cols[1:]:
            table = tables[col[0]]
            db_schema[db].append(('{}.{}'.format(db, table), 'has', '{}.{}.{}'.format(db, table, col[1])))

In [5]:
schema_graph = '/home/22842219/Desktop/openSource/CompGCN/data/Cy-Spider/schema_graph.json'
with codecs.open(schema_graph, 'r', 'utf-8-sig') as f:
    triples  ={}
    nodes_lookup = {}
    schema_data = json.load(f)
    nodes = schema_data[0]['nodes']
    for node in nodes:
        nodes_lookup[node['identity']]=node['labels']
    rels = schema_data[0]['relationships']
    for each in rels:
        start = nodes_lookup[each['start']][0]
        end = nodes_lookup[each['end']][0]
        db = start.split('.')[0]
        if db not in triples:
            triples[db]=[]
        triples[db].append((start, each['type'], end))

In [6]:
triples

{'device': [('device.device', 'device.stock', 'device.shop')],
 'cre_Doc_Control_Systems': [('cre_Doc_Control_Systems.Draft_Copies',
   'cre_Doc_Control_Systems.draft_copies_HAS_cre_Doc_Control_Systems.document_drafts',
   'cre_Doc_Control_Systems.Document_Drafts'),
  ('cre_Doc_Control_Systems.Documents',
   'cre_Doc_Control_Systems.documents_HAS_cre_Doc_Control_Systems.ref_document_status',
   'cre_Doc_Control_Systems.Ref_Document_Status'),
  ('cre_Doc_Control_Systems.Documents',
   'cre_Doc_Control_Systems.documents_HAS_cre_Doc_Control_Systems.ref_document_types',
   'cre_Doc_Control_Systems.Ref_Document_Types'),
  ('cre_Doc_Control_Systems.Document_Drafts',
   'cre_Doc_Control_Systems.document_drafts_HAS_cre_Doc_Control_Systems.documents',
   'cre_Doc_Control_Systems.Documents'),
  ('cre_Doc_Control_Systems.Addresses',
   'cre_Doc_Control_Systems.Documents_Mailed',
   'cre_Doc_Control_Systems.Documents'),
  ('cre_Doc_Control_Systems.Documents',
   'cre_Doc_Control_Systems.documents_

In [7]:
train_split = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/train.json'
dev_split = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/dev.json'

for i, split in enumerate([train_split, dev_split]):
    split_dbs = []
    with codecs.open(split, 'r', 'utf-8-sig') as f:
        split_data = json.load(f)
        for each in split_data:
            db = each['db_id']
            if db not in split_dbs:
                split_dbs.append(db)
    split_schema  =[]
    for db in split_dbs:
        if db in triples:
            split_schema.extend(triples[db])  
    for db in db_schema:
        if db in db_schema:
            split_schema.extend(db_schema[db])
    import os
    fpath= os.getcwd() +  '/{}.txt'.format(i)
    print(fpath)
    with open(fpath, 'w') as out:
        for item in split_schema:
            out.write('\t'.join(str(element) for element in item) + '\n')

/home/22842219/Desktop/openSource/CompGCN/data/Cy-Spider/0.txt
/home/22842219/Desktop/openSource/CompGCN/data/Cy-Spider/1.txt


In [8]:
os.getcwd()

'/home/22842219/Desktop/openSource/CompGCN/data/Cy-Spider'

In [11]:

ent_set, rel_set =set(), set()
for split in ['train', 'valid']:
    for line in open('{}.txt'.format( split)):
        sub, rel, obj = map(str.lower, line.strip().split('\t'))
        print(sub, rel, obj)
        ent_set.add(sub)
        rel_set.add(rel)
        ent_set.add(obj)

ent2id = {ent: idx for idx, ent in enumerate(ent_set)}

department_management.head department_management.management department_management.department
farm.farm farm.competition_record farm.farm_competition
farm.farm_competition farm.farm_competition_has_farm.city farm.city
student_assessment.people_addresses student_assessment.people_addresses_has_student_assessment.addresses student_assessment.addresses
student_assessment.candidate_assessments student_assessment.candidate_assessments_has_student_assessment.candidates student_assessment.candidates
student_assessment.candidates student_assessment.candidates_has_student_assessment.people student_assessment.people
student_assessment.students student_assessment.students_has_student_assessment.people student_assessment.people
student_assessment.people_addresses student_assessment.people_addresses_has_student_assessment.people student_assessment.people
book_2.publication book_2.publication_has_book_2.book book_2.book
musical.actor musical.actor_has_musical.musical musical.musical
twitter_1.user_pr

In [12]:

with open('ent2id_vocabulary.txt', 'w') as out:
    for ent, idx in ent2id.items():
        out.write(ent + '\n')